<a href="https://colab.research.google.com/github/Pyfin5/Machine_Learning/blob/main/Compliance_Comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import requests,  zipfile, io
from urllib.request import urlretrieve
import pandas as pd
import openpyxl
import json
import os
from dotenv import load_dotenv
from datetime import datetime
import re
from google.genai.types import GenerateContentConfig, Part
from pydantic import BaseModel, Field
from google import genai
from google.genai import types
from IPython.display import display, Markdown
from google.colab import userdata

In [7]:


print('Environment variables laded')


PDF_MIME_TYPE = "application/pdf"

client = genai.Client(
    api_key = userdata.get('GOOGLE_AI_STUDIO_API_KEY'),
    )

system_prompt = f"""You are a detail oriented regulatory analyst.
    You will be given a set of documents. Your task is to extract
    relevant information based on user prompts with a focus on
    providing differences between the two document. The documents
    will be viewed from a third party risk perspective"""

comparison_item = "Criticality"

# Prompt is formatted to use the 'comparison item' variable in the prompt
comparison_prompt = f"""Compare the two documents provided. The first document is from the
European Banking Authority (EBA) and the second is from the Office of the Superintendent of Financial Institutions (OSFI).
Focus on identifying key differences in approaches to the term '{comparison_item}', how it is defined, requirements,
applications and any other relevant aspects. Provide a detailed comparison"""





Environment variables laded


In [9]:
# Send Comparison Prompt to Gemini
response = client.models.generate_content(
    model="gemini-2.5-pro",
    contents=[
        comparison_prompt,
        Part.from_uri(
            file_uri="https://www.eba.europa.eu/sites/default/files/2025-07/33a0ee15-9601-4c2b-828e-1b09201a6e9f/CP%20on%20Draft%20Guidelines%20on%20sound%20management%20of%20third%20party%20risk.pdf",
            mime_type=PDF_MIME_TYPE,
        ),
        Part.from_uri(
            file_uri="https://www.osfi-bsif.gc.ca/en/print/pdf/node/567",
            mime_type=PDF_MIME_TYPE,
        ),
    ],
    config=GenerateContentConfig(
        temperature=0,
        system_instruction=system_prompt
        ),
)



In [10]:
display(Markdown(f"### Comparison"))
display(Markdown(response.text))

### Comparison

Of course. As a detail-oriented regulatory analyst, I will provide a detailed comparison of the European Banking Authority (EBA) and the Office of the Superintendent of Financial Institutions (OSFI) documents, focusing on the key differences in their approaches to the term 'Criticality' from a third-party risk perspective.

### **Executive Summary**

The primary difference between the EBA and OSFI approaches to 'Criticality' lies in their fundamental philosophy. The **EBA** adopts a prescriptive, function-centric approach, using the combined term **'critical or important function'**. This classification is a distinct gateway that, once triggered, subjects the arrangement to a specific and stricter set of requirements. The determination is heavily based on the direct consequences of a function's failure on the financial entity's performance, compliance, and continuity.

In contrast, **OSFI** employs a more principles-based, conceptual approach. It defines **'Criticality'** as a measure of *importance* based on *impact*, explicitly stating it is "irrespective of the likelihood of such risk event occurring." For OSFI, criticality is a key input into a broader, scalable risk assessment rather than a simple binary classification. It is used to determine the *intensity* with which a Federally Regulated Financial Institution (FRFI) must apply the guideline's principles.

While both regulators use the concept to apply heightened scrutiny, the EBA's framework is akin to a checklist with clear triggers and consequences, whereas OSFI's framework provides guiding principles for a more holistic, risk-based judgment by the institution itself.

---

### **Detailed Comparison of 'Criticality'**

Here is a detailed breakdown of the differences in definition, requirements, application, and other relevant aspects.

#### **1. Definition and Terminology**

| Aspect | **EBA (European Banking Authority)** | **OSFI (Office of the Superintendent of Financial Institutions)** |
| :--- | :--- | :--- |
| **Primary Term** | **Critical or important function** | **Criticality** |
| **Core Definition** | A function is defined by the *consequence of its failure*. "means a function, the disruption of which would materially impair the financial performance of a financial entity, or the soundness or continuity of its services and activities, or the discontinued, defective or failed performance of that function would materially impair the continuing compliance of a financial entity with the conditions and obligations of its authorisation, or with its other obligations under applicable financial services law." (EBA, p. 21) | A concept defined by *importance and impact*, irrespective of probability. "'Criticality' denotes importance to the FRFI's operations, strategy, financial condition or reputation. It emphasizes the impact of a risk event, irrespective of the likelihood of such risk event occurring." (OSFI, p. 5) |
| **Contextual Nuance** | The EBA explicitly notes that its definition is aligned with other EU regulations like DORA and MiFID II but is distinct from the definition of 'critical functions' used for bank resolution purposes (BRRD), although it encompasses them. (EBA, p. 10, para 17) | OSFI links criticality directly to the concept of operational resilience and defines related terms like 'Critical operations' and 'Critical third-party arrangements' to provide a holistic view of what is important to the institution's continued operation. (OSFI, p. 5) |

**Key Differences in Definition:**

*   **Function vs. Concept:** The EBA defines a *type of function* ('critical or important'). OSFI defines a *concept* ('Criticality') that is then applied to arrangements and operations.
*   **Focus of Definition:** The EBA's definition is purely consequential—it is critical *if* its failure causes material impairment. OSFI's definition is broader, encompassing importance to strategy and reputation, and explicitly decouples impact from likelihood.

---

#### **2. Assessment and Determination Criteria**

| Aspect | **EBA (European Banking Authority)** | **OSFI (Office of the Superintendent of Financial Institutions)** |
| :--- | :--- | :--- |
| **Assessment Approach** | **Prescriptive with automatic triggers.** The guidelines provide specific scenarios where a function must be considered critical or important. | **Principles-based and scalable.** Criticality is an input into a scalable risk assessment. The FRFI is expected to use the criteria to make a judgment. |
| **Specific Criteria** | The EBA provides a multi-layered set of criteria: <br> **1. Automatic Classification:** A function is *always* considered critical or important if its disruption materially impairs: <br>    - Compliance with authorization/legal obligations. <br>    - Financial performance. <br>    - Soundness or continuity of services. (p. 28, para 33) <br> **2. Presumptive Classification:** <br>    - Operational tasks of internal control functions (unless proven otherwise). (p. 28, para 34) <br>    - Functions requiring regulatory authorization (e.g., banking activities, payment services). (p. 28, para 35) <br> **3. Factor-Based Assessment:** A list of factors must be considered, including: <br>    - Connection to core service provision. <br>    - Impact on financial resilience, business continuity, operational/legal/reputational/ESG risk. <br>    - Impact on resolvability and recovery planning. (p. 29, para 37) | OSFI provides a more concise set of guiding factors for the FRFI to consider when determining the level of criticality: <br> - The severity of loss or harm to the FRFI if the third party fails. <br> - Substitutability of the third party, including portability and timeliness of transfer. <br> - The degree to which the third party supports a 'critical operation' of the FRFI. <br> - The impact on business operations if an exit were required. (OSFI, p. 10, sec 2.1.1) |

**Key Differences in Assessment:**

*   **Prescription vs. Principle:** The EBA's approach is more directive, with several "always consider" and "automatically consider" triggers. This reduces the level of judgment required by the financial entity. OSFI provides principles that the FRFI must interpret and apply, allowing for more flexibility but also requiring a more robust internal rationale.
*   **Scope of Factors:** The EBA's list of factors is more extensive and granular, explicitly including items like ESG risk and resolvability, reflecting the broader EU regulatory landscape. OSFI's factors are higher-level and focused on core operational impacts like substitutability and severity of harm.

---

#### **3. Application and Consequential Requirements**

| Aspect | **EBA (European Banking Authority)** | **OSFI (Office of the Superintendent of Financial Institutions)** |
| :--- | :--- | :--- |
| **Regulatory Impact** | **Binary Trigger for Stricter Rules.** If a function is classified as 'critical or important', a specific, more stringent set of requirements is automatically applied. | **Scaling Factor for Intensity.** Criticality is used to scale the intensity of risk management. For 'high-risk and critical' arrangements, the entire guideline becomes a set of *minimum expectations*, and more rigorous oversight is required. |
| **Specific Requirements** | For 'critical or important functions', the EBA mandates: <br> - **Stricter Contractual Clauses:** Additional, specific clauses are required in the written agreement (p. 46, para 86). <br> - **Mandatory Audit Rights:** Unrestricted access and audit rights for the entity and its regulators are non-negotiable (p. 50, para 99). For non-critical functions, these rights are risk-based. <br> - **Subcontracting Oversight:** Specific conditions must be met and documented for any subcontracting (chain outsourcing) (p. 47, sec 12.1). <br> - **Documented Exit Strategy:** A formal, documented exit plan is mandatory (p. 55, para 117). <br> - **Enhanced Reporting:** More detailed information must be maintained in the register of third-party arrangements (p. 38, para 64) and competent authorities must be informed of planned arrangements (p. 39, para 67). | For arrangements deemed 'critical' (and high-risk), OSFI requires: <br> - **More Rigorous Due Diligence:** A non-exhaustive list of due diligence factors is provided in Annex 1 that must be considered (p. 26). <br> - **Comprehensive Contract Provisions:** Annex 2 provides a list of provisions that should be included in agreements (p. 28). <br> - **Formal Contingency/Exit Plans:** Documented plans with specific elements (e.g., triggers, playbooks for stressed/non-stressed exits) are required (p. 20, sec 2.3.5.1). <br> - **Heightened Monitoring:** More frequent and rigorous assessment and monitoring are expected (p. 10). |

**Key Differences in Application:**

*   **"If-Then" Logic vs. Sliding Scale:** The EBA's framework operates on a clear "if-then" basis: if the function is critical, then specific, enumerated rules apply. OSFI's framework is more of a sliding scale: the more critical the arrangement, the more robustly and intensely *all* principles in the guideline must be applied.
*   **Clarity of Distinction:** The EBA creates a very clear line between requirements for critical and non-critical functions (e.g., mandatory vs. risk-based audit rights). In the OSFI guideline, the expectations apply broadly, but their *rigor* is scaled based on risk and criticality. The entire guideline effectively becomes the "stricter set of rules" for critical arrangements.